In [1]:
from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb

import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = DefaultArgsNamespace()

train_loader, val_loader, test_loader = eee_get_dataloaders(args)

# # get input feature  size
# dataiter = next(iter(train_loader))
# batch =  dataiter


# # Access the parsed arguments
model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

dummy_input = torch.randn(1, 30, 1024) # batch, num_frames, channels, height, width

dummy_input = dummy_input.to(args.device)

dummy_output = model(dummy_input)   

print(dummy_output.shape)




train dataset size:  5686
val dataset size:  1892
test dataset size:  2477


/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 16])


In [2]:
keysteps = args.dataloader_params['keysteps']
out_classes = len(keysteps)
print(out_classes)

31


In [6]:
modality = args.dataloader_params['modality']
print("Modality: ", modality)

# Find feature dimension
input, feature_size, label = preprocess(next(iter(train_loader)), args.dataloader_params['modality'], model.extract_resnet, device=device)
print("Feature size: ", feature_size)

print(input[:,0,-3:])

args.transformer_params['input_dim'] = feature_size
args.transformer_params['output_dim'] = out_classes

model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

wandb_logger = wandb.init(
    # set the wandb project where this run will be logged
    project="EgoExoEMS",
    group="Keystep Recognition",
    mode="disabled",
    name="Testing on EgoExoEMS with I3D RGB Features - ICRA Model",
    notes="initial attempt ICRA model with I3D RGB features",
    config={
    "args": args,
    }
)



Modality:  ['flow', 'rgb', 'smartwatch']
Feature size:  2051
tensor([[-1.0139, -0.3012,  1.4067]], device='cuda:0')


In [8]:
# # print one batch
for i, batch in enumerate(train_loader):
    input,feature_size, label = preprocess(batch, args.dataloader_params['modality'], model.extract_resnet, device=device)
    print("Model input:",input.shape)
    output = model(input)
    print("Model output:",output.shape)
    print("Label:",label)
    break

# results_dir = './results'
# # # # Train the model
# for epoch in range(args.learning_params["epochs"]):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, wandb_logger, modality=modality)
#     val_loss = validate(model, val_loader, criterion, device, wandb_logger, modality=modality)
#     scheduler.step()
#     print(f"Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")
#     results = test_model(model, test_loader, criterion, device, wandb_logger, modality=modality, epoch=epoch, results_dir=results_dir)
#     print(results)
#     break


Model input: torch.Size([1, 30, 2051])
Model output: torch.Size([1, 31])
Label: tensor([4], device='cuda:0')
